## Technical Question Answerer With Gradio
- Ask a technical question to a chatbot embued with multimodal capabilities.
- Choose between different models (e.g. OpenAI's GPT, Anthropic's Claude).

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic

import gradio as gr
import base64
from io import BytesIO
from PIL import Image
from IPython.display import Audio, display

# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

# Connect to OpenAI, Anthropic and Google; comment out the Claude or Google lines if you're not using them
openai = OpenAI()
claude = anthropic.Anthropic()
# google.generativeai.configure()

system_message = "You are a helpful assistant that explains technical contents and responds in markdown"

def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=message)

    audio_stream = BytesIO(response.content)
    output_filename = "output_audio.mp3"
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())

    display(Audio(output_filename, autoplay=True))


In [ ]:
# --- Chat functions ---
def chat_gpt(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    reply = response.choices[0].message.content
    history = history + [{"role": "assistant", "content": reply}]
    talker(reply)   # make it talk
    return history


claude_via_openai_client = OpenAI(
    api_key=anthropic_api_key, 
    base_url="https://api.anthropic.com/v1"
)

def chat_claude(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = claude_via_openai_client.chat.completions.create(
        model="claude-3-haiku-20240307",
        messages=messages
    )
    reply = response.choices[0].message.content
    history = history + [{"role": "assistant", "content": reply}]
    talker(reply)   # make it talk
    return history


# --- Gradio UI ---
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
    with gr.Row():
        the_model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history, model):
        # add user turn
        history = history + [{"role": "user", "content": message}]
        # call selected model
        if model == "GPT":
            history = chat_gpt(history)
        elif model == "Claude":
            history = chat_claude(history)
        return "", history

    entry.submit(
        fn=do_entry,
        inputs=[entry, chatbot, the_model],
        outputs=[entry, chatbot]   # only 2 outputs
    )

    clear.click(lambda: [], None, chatbot, queue=False)

In [ ]:
ui.launch(inbrowser=True)

# prompt = """
# Please explain what this code does and why:
# yield from {book.get("author") for book in books if book.get("author")}
# """